## Reed-Muller Code:

In [3]:
class ReedMullerCode:
    
    def __init__(self, r, m):
            
        self.r = r
        self.m = m
        self.q = 2
        
        self.n = 2**self.m
        self.k = sum(binomial(self.m,i) for i in range(self.r + 1))
        self.d = 2**(self.m - self.r)
        
        # Initializing field
        self.F = FiniteField(self.q)
        
        self.G = matrix(self.F, self.k, self.n, lambda i,j : 0)
        
        print('RM({0},{1}) has the parameters:'.format(self.r, self.m))
        print('n, k, d:')
        print(self.n,self.k,self.d)
        self.G = self.get_G(self.r, self.m)
        print('and generator matrix:')
        print(self.G)
        
    
    def get_G(self, r, m):
        n = 2**m
        k = sum(binomial(m,i) for i in range(r + 1))
        d = 2**(m - r)

        #if (n == k):
            # trivial code

        if (n == d):
            # repetition code
            return matrix(self.F, k, n, lambda i,j : 1)

        elif (d == 2 and n == k + 1):
            # parity check code
            PCC = codes.ParityCheckCode(GF(self.q), k)
            return PCC.generator_matrix()

        else:
            return block_matrix([ [self.get_G(r,m-1), self.get_G(r,m-1)], [0, self.get_G(r-1,m-1)] ])
    
    
    def Encoding(self, message, zeropad = True):
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.F(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunk(message[i:i+self.k]))
        
        return vector(self.F, c)
            
    def EncodeChunk(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G

        return c
        
    
    def Decoding(self, received):
        
        # Check input size
        if len(received) % self.n != 0:
            raise ValueError('Invalid input size')
            
        c = []
        
        for i in range(0,len(received),self.n):
            c.extend(self.DecodeChunk(received[i:i+self.n], self.r, self.m))
            
        return vector(self.F, c)
                
            
    def DecodeChunk(self, word, r, m):
        
        word = vector(self.F, word)
        
        if (self.n % len(word)) != 0:
            raise ValueError('Invalid chunk size')
        
        if (r == 0):
            return self.repetition_decode(word)
        
        if (m == r + 1):
            return 'parity check'
        
        l = len(word)
        u1 = word[:l//2]
        u2_v = word[l//2:]
        
        v_error = u1 + u2_v

        v = self.DecodeChunk(v_error, r - 1, m - 1)
        u2 = u2_v + v
        
        u1_decoded = self.DecodeChunk(u1, r, m - 1)
        u2_decoded = self.DecodeChunk(u2, r, m - 1)
        u = 0
        
        if (u1_decoded == 'parity check' and u2_decoded == 'parity check'):
            u = self.parity_check_compare(u1, u2)
            return vector(self.F, list(u) + list(u + v))
        else:
            return vector(self.F, list(u1_decoded) + list(u2_decoded + v))
        
    
    
    def repetition_decode(self, word):
        
        if word.hamming_weight() >= len(word.list())//2:
            return vector(self.F, [1 for i in range(len(word.list()))])
        elif word.hamming_weight() < len(word.list())//2:
            return vector(self.F, [0 for i in range(len(word.list()))])
        else:
            return "decoding failure"
        
    def parity_check_compare(self, word1, word2):
        decoded1 = word1[:-1]
        decoded2 = word2[:-1]
        
        if (((decoded1.hamming_weight() % 2) == 0 and word1[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word1[-1] == 1)):
            return word1
        elif (((decoded2.hamming_weight() % 2) == 0 and word2[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word2[-1] == 1)):
            return word2
        else:
            return "decoding failure"
        

In [5]:
C = ReedMullerCode(1,3)

RM(1,3) has the parameters:
n, k, d:
8 4 4
and generator matrix:
[1 0 0 1|1 0 0 1]
[0 1 0 1|0 1 0 1]
[0 0 1 1|0 0 1 1]
[-------+-------]
[0 0 0 0|1 1 1 1]


In [2]:
message = vector(GF(2), [1,0,1,0,1,])
print('message:\n', message)

c = C.Encoding(message)
print('codeword:\n', c)

e = vector(GF(2), [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0])

r = c + e
print('recevied word:\n', r)

decoded = C.Decoding(r)
print("decoded word\n", decoded)

message:
 (1, 0, 1, 0, 1)


NameError: name 'C' is not defined

In [188]:
C.G

[1 0 0 1 0 1 1 0|0 1 1 0 1 0 0 1]
[0 1 0 1 0 1 0 1|0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1|0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1|0 0 0 0 1 1 1 1]
[---------------+---------------]
[0 0 0 0 0 0 0 0|1 1 1 1 1 1 1 1]

In [176]:
C.G.echelon_form()

[1 0 0 1 0 1 1 0|0 1 1 0 1 0 0 1]
[0 1 0 1 0 1 0 1|0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1|0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1|0 0 0 0 1 1 1 1]
[---------------+---------------]
[0 0 0 0 0 0 0 0|1 1 1 1 1 1 1 1]

In [181]:
A = matrix(GF(2), 4,8, [1,1,0,0, 1,1,1,1, 0,1,0,0, 1,0,1,1,0,0,1,0, 1,1,0,1, 0,0,1,1, 1,1,1,0])

In [183]:
A.echelon_form()

[1 0 0 0 0 1 0 0]
[0 1 0 0 1 0 1 1]
[0 0 1 0 1 1 0 1]
[0 0 0 1 0 0 1 1]

In [187]:
A = matrix(GF(2), 5, 16, 0)
A = C.G
A[3][0]

0

In [190]:
C.G.rank()

5

In [191]:
C.G.echelon_form(transformation=true)

[1 0 0 1 0 1 1 0|0 1 1 0 1 0 0 1]
[0 1 0 1 0 1 0 1|0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1|0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1|0 0 0 0 1 1 1 1]
[---------------+---------------]
[0 0 0 0 0 0 0 0|1 1 1 1 1 1 1 1]

In [194]:
C.G.echelonize(); C.G

[1 0 0 1 0 1 1 0|0 1 1 0 1 0 0 1]
[0 1 0 1 0 1 0 1|0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1|0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1|0 0 0 0 1 1 1 1]
[---------------+---------------]
[0 0 0 0 0 0 0 0|1 1 1 1 1 1 1 1]

In [195]:
c1 = c[:5]
c1

(1, 0, 1, 0, 1)

In [225]:
G1 = C.G.submatrix(0,0,5,16)
rank(G1)
print(G1.echelon_form())

[1 0 0 1 0 1 1 0 0 1 1 0 1 0 0 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]


In [230]:
B=G1.change_ring(QQ)
sage: B.echelon_form()

[1 0 0 1 0 1 1 0 0 1 1 0 1 0 0 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]

In [229]:
G1.rref()

[1 0 0 1 0 1 1 0 0 1 1 0 1 0 0 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]

In [231]:
RREF_binary(G1)

NameError: name 'RREF_binary' is not defined

In [234]:
test = identity_matrix(GF(2), 5)

In [236]:
test.inverse()

[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]

In [ ]:
10110